In [3]:
import modules.swmm_ex.swmm_ex_multiprocessing as ex
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os
from modules.extract_events import extract_events

# Set relevant data paths
events_path = '02_input_data\\events_FMO.csv'
P_path = '02_input_data\\P_FMO.csv'
save_folder = '02_input_data\\events_FMO'

# Read in data
P_events = pd.read_csv(events_path)
P_series = pd.read_csv(P_path)


P_sample = P_events[P_events['hN_mm'] < 2].sample(n=10)

P_sample

,start,end,hN_mm,iN_mean,iN_max,hN_max_60,DN_h
1034,2019-11-29 15:00:00,2019-11-29 15:25:00,0.64,0.106667,0.31,0.00,0.416667
1506,2022-10-16 20:15:00,2022-10-16 21:00:00,0.87,0.087000,0.30,0.00,0.750000
435,2016-04-27 14:30:00,2016-04-27 17:20:00,0.40,0.011429,0.11,0.32,2.833333
1117,2020-05-03 15:10:00,2020-05-03 17:20:00,1.63,0.060370,0.61,1.41,2.166667
1674,2023-10-09 05:45:00,2023-10-09 05:55:00,0.09,0.030000,0.06,0.00,0.166667
248,2015-06-17 22:25:00,2015-06-18 04:15:00,1.58,0.022254,0.18,0.68,5.833333
1503,2022-10-08 03:40:00,2022-10-08 04:25:00,0.79,0.079000,0.25,0.00,0.750000
1014,2019-10-17 18:10:00,2019-10-17 20:05:00,0.80,0.033333,0.27,0.66,1.916667
1283,2021-05-11 11:35:00,2021-05-11 13:10:00,0.63,0.031500,0.14,0.34,1.583333
1595,2023-03-30 07:15:00,2023-03-30 07:30:00,0.36,0.090000,0.30,0.00,0.250000


In [ ]:
extract_events(P_sample, P_series, save_folder)